In [63]:
import os
os.chdir('/content/drive/MyDrive/000GithubRepos/bigcontest2024')

In [64]:
pip install python-dotenv

In [65]:
pip install langchain_community

In [66]:
pip install sentence-transformers

In [67]:
pip install neo4j

In [68]:
pip install neo4j_genai

In [69]:
pip install openai

In [70]:
pip install langchain_huggingface

In [71]:
pip install langchain_google_genai

In [72]:
pip install openai

In [96]:
pip install streamlit

In [73]:
from llm_response.get_llm_model import get_llm_model

llm = get_llm_model()

In [74]:
from cypher_query.retrieval_query import retrievalQuery, retrievalQuery_v2
from dotenv import load_dotenv
from config import CONFIG

from graphrag.get_embedding_model import get_embedding_model
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

load_dotenv()

True

In [75]:
os.getenv('NEO4J_URI')

'bolt://43.201.102.234/:7687'

In [76]:
from cypher_query.retrieval_query import retrievalQuery, retrievalQuery_v2, retrievalQuery_v3

In [77]:
from utils import graphdb_driver

In [78]:
def get_neo4j_vector(index_name='queryVector'):
    neo4jvector = Neo4jVector.from_existing_index(
        embedding=get_embedding_model(),  # Using the custom embedding function
        url=CONFIG.neo4j_url,
        database='neo4j',
        username=CONFIG.neo4j_user,
        password=CONFIG.neo4j_password,
        index_name=index_name,
        text_node_property="textEmbedding",
        retrieval_query=retrievalQuery_v3
    )
    return neo4jvector

def retrieve_store_nodes(query):
    """
    쿼리 임베딩 <-> 리뷰 임베딩 유사도 측정하여 상위 n개의 Store 노드 추출
    """
    store_retriever = get_neo4j_vector().as_retriever(search_kwargs={"k": 6})
    result_nodes = store_retriever.invoke(query)  # invoke는 동기적으로 실행되는 메서드
    return result_nodes

store_retriever = get_neo4j_vector().as_retriever(search_kwargs={"k": 5})
vs = get_neo4j_vector()

ERROR:neo4j.io:Failed to write data to connection IPv4Address(('43.201.102.234', 7687)) (ResolvedIPv4Address(('43.201.102.234', 7687)))


In [79]:
query = "8살 아이 포함 3인 가족 가기 좋은 평균가격 3만원대 패밀리 레스토랑 추천"
query = "20대 후반 여자 또래 친구끼리 가기 좋은 카페 추천해줘"
query = '8살 아이 포함 3인 가족 가기 좋은 평균가격 3만원대 패밀리 레스토랑 추천해줘'
query = "제주 중문 근처에서 30대 혼자 여행객에게 적합한 1~2만원대 한식 메뉴를 제공하는 조용한 맛집을 추천해 주세요. 혼자 방문하기 편안한 곳이면 좋겠습니다."

In [80]:
store_retriever_rev_emb = get_neo4j_vector().as_retriever(search_kwargs={"k": 10})

ERROR:neo4j.io:Failed to write data to connection IPv4Address(('43.201.102.234', 7687)) (ResolvedIPv4Address(('43.201.102.234', 7687)))


In [81]:
rev_sim_result = store_retriever_rev_emb.invoke(query)

ERROR:neo4j.io:Failed to write data to connection IPv4Address(('43.201.102.234', 7687)) (ResolvedIPv4Address(('43.201.102.234', 7687)))


In [82]:
rev_sim_result

[Document(metadata={'storeName': '알리올레', 'store_Image': {'kakao': 'https://t1.kakaocdn.net/thumb/T800x0.q50/?fname=http%3A%2F%2Ft1.kakaocdn.net%2Fmystore%2FF25E55EE0C024405AC7DF6C34928C5B7', 'google': 'https://lh5.googleusercontent.com/p/AF1QipPit1Qw8VC2aE8b_X8CBkObP64f4-JeXB7EreXq=w408-h544-k-no', 'naver': 'https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230709_216%2F1688828828008vDHWP_JPEG%2F1683114915301.jpg'}, 'reviewText': '제주 혼자 여행왔는데 숙소 근처에 맛집이 있어서 소개합니다 ^^ 맛도 분위기도 친절함도\n힐링 그 자체!', 'menu': '한치 모닥카츠:14000, 치즈 모닥카츠:15000, 등심&안심 모닥카츠:14000, 새우 모닥카츠:14000', 'store_Rating': {'kakao': 4.8, 'google': 0, 'naver': None}, 'wait_time': '{"30분 이내":2,"바로 입장":97}', 'purpose': '여행, 데이트, 나들이', 'viwit_with': '연인・배우자, 아이, 친구', 'use_how': '{"예약 없이 이용":96,"예약 후 이용":3}', 'store_Addr': '제주 제주시 구좌읍 송당리 1398-7번지 가동 1층', 'reviewCount': {'kakao': 22, 'google': 0, 'naver': None}, 'store_Type': '양식', 'pk': 4977}, page_content='제주 혼자 여행왔는데 숙소 

In [120]:
records, summary, keys = graphdb_driver.execute_query("""
// 1. '제주시'에 속한 지역 중 '해변'을 포함하는 관광지(ATTR)가 있는 지역 찾기
MATCH (c:City)-[:HAS_REGION]->(rg:Region)-[:HAS_ATTR]->(a:ATTR)
WHERE c.name = "제주시" AND a.AREA_NM CONTAINS "해변"
WITH rg, a, a.latitude AS attrLat, a.longitude AS attrLon

// 2. 해당 지역 내에서 가정식 가게 찾기 및 가격 필터링
MATCH (rg)-[:HAS_STORE]->(s:STORE)
WHERE s.MCT_TYPE = "가정식"
AND ANY(menuItem IN split(s.menu, ", ")
        WHERE toInteger(apoc.text.regreplace(menuItem, ".*:(\\d+)", "$1")) >= 40000
        AND toInteger(apoc.text.regreplace(menuItem, ".*:(\\d+)", "$1")) < 50000)

// 3. 방문 대상 조건 필터링
MATCH (s)-[:HAS_VISIT_KEYWORD]->(v:Visit_with)
WHERE v.member CONTAINS "부모님" OR v.member CONTAINS "아이"

// 4. 거리 계산 및 결과 출력
WITH s, v, attrLat, attrLon, s.latitude AS storeLat, s.longitude AS storeLon
RETURN s.pk AS pk, s.MCT_NM AS RestaurantName, s.ADDR AS Address, s.menu AS Menu,
       v.member AS VisitWith,
       toInteger(point.distance(point({latitude: attrLat, longitude: attrLon}), point({latitude: storeLat, longitude: storeLon}))) AS Distance_in_meters_from_Beach
ORDER BY Distance_in_meters_from_Beach ASC
LIMIT 50
""")

In [121]:
# pk 기준 중복 제거
records_drop_dup = []
for r in records:
    if r['pk'] not in [d['pk'] for d in records_drop_dup]:
        records_drop_dup.append(r)

In [122]:
len(records_drop_dup)

27

In [123]:
query = '40대 부모와 10대 아이가 함께 먹을 수 있는 제주시 해변 근처 4~5만원대 한식당 추천해주세요. 가족이 함께 가기 편안한 곳이면 좋겠어요'

In [124]:
embedding_model = get_embedding_model()
query_embedding = embedding_model.embed_query(query)

In [125]:
from llm_response.langgraph_nodes.recommendation.get_store_candidates import retrieve_top_k_reviews

In [126]:
candidate_str = ''
t2c_candidates_cnt = 0
for r in records_drop_dup:
    if t2c_candidates_cnt == 3:
        break
    r_keys = r.keys()
    one_record_str = ''
    for key in r_keys:
        one_record_str += f"{key} : {str(r[key])[:100]}\n"
        if key == 'pk':
            reviews = retrieve_top_k_reviews(r[key], query_embedding, graphdb_driver, k=2)
            if reviews:
                reviews_lst = [f"{ri}. {review['text'][:100]}" for ri, review in enumerate(reviews, start=1)]
                one_record_str += f"리뷰 : {', '.join(reviews_lst)}\n"
    if '리뷰' in one_record_str:
        t2c_candidates_cnt += 1
        candidate_str += one_record_str + '\n'

In [127]:
print(candidate_str)

pk : 2993
리뷰 : 1. 화요일정기휴무래요. 우럭튀김(우럭정식)2인3만원 가격올랐어요. 22.4.9기준 탐나는전 안됩니다.2년전 왔을때보다 맛이 별로인듯요.ㅎㅎ ㅜㅜ 좀 짜기만하다? ㅠㅠ 사장님이 나이가 , 2. 제주에서 첫끼였는데 맛있어요 첫번째 단추를 잘껴서인지 먹으러가는곳마다  다 맛있었어요 우럭구이  대박 바삭바삭소스도굿~사장님친절
월정리해수욕장도  굿~너무좋았어요
RestaurantName : 바당지기
Address : 제주 제주시 구좌읍 월정리 30번지 1층
Menu : 우럭정식 2인기준:34000, 전복죽 2인기준:30000, 우럭조림 2인기준:40000, 고등어조림 2인기준:30000, 매운탕 2인기준:40000, 뿔소라구이:30000, 갈치조
VisitWith : 아이
Distance_in_meters_from_Beach : 74

pk : 7151
리뷰 : 1. 고기를 손수 다 구워주셨어요~ 맛은 말해모해요^^, 2. 음식은 훌륭하고 양이 많았으나 4명이 식사하기에는 테이블이 조금 작았습니다.
RestaurantName : 제주흑돼지마씸
Address : 제주 제주시 삼도이동 1261-19번지 1층
Menu : 흑돼지고기:30000, 흑돼지근고기 2인분:60000, 백돼지근고기 2인분:49000, 백돼지근고기 1인분:24500, 김치찌개 대:13000, 김치찌개 소:7000, 된장찌개:7
VisitWith : 아이
Distance_in_meters_from_Beach : 127

pk : 8550
리뷰 : 1. 절대 가지마세요.
전복뚝배기에 바지락은 해감이 전혀 안되있어서 다들 한두개만 먹고 말았어요. 항의하니까 12시간 해감해도 어쩔수 없다고 하시네요. 식당에서 해감이 안된 걸 내오는데, 2. 제주도 지인 친구가 소개 시켜줘서 평대해변갔다가
방문했어요. 실내 공간도 청결하고 깨끗하고
전복회와 전복구이도 흡입 하 듯이 먹고 왔습니다!
전복 돌솥밥의 특제 간장소스를 비벼 먹
RestaurantName : 평대전복
Address : 제주 